## Introduction

This script effectively retrieves river gauge numbers from a specified webpage using Python's requests and BeautifulSoup libraries, enabling easy access to this data for further analysis or processing.

In [2]:
# import the necessary packages
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd


## Retrieve a list of all available river gauge numbers from the website.

In [3]:
# Retrieve all river gauge numbers (pegelnummern) as a list

# URL of the webpage containing river gauge data
url = "https://www.pegelonline.wsv.de/gast/pegeltabelle?scrollPosition=0&gewaesserFilter=16&pegelnameFilter=-&pegelnummerFilter=-&parameterFilter=3"

# Make a GET request to fetch the webpage content
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract all river gauge numbers from the text and convert them to integers
    pegelnummern = [int(num.strip()) for num in soup.get_text().split('\n') if num.strip().isdigit()]

    # Print the list of river gauge numbers
    print("All river gauge numbers (Pegelnummern):")
    print(pegelnummern)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


All river gauge numbers (Pegelnummern):
[10046105, 10054500, 10056302, 10061007, 10062000, 10068006, 10078000, 10081004, 10088003, 10089006, 10090408, 10090708, 10091008, 10092000, 10094006, 10096001, 13101016, 13409200, 166640, 2310010, 23300130, 23300320, 23300580, 23300800, 23300900, 23500600, 23500700, 23700200, 23700500, 23700600, 23700700, 23800100, 23800120, 23800140, 23800145, 23800176, 23800180, 23800200, 23800220, 23800240, 23800260, 23800280, 23800300, 23800320, 23800322, 23800360, 23800400, 23800420, 23800440, 23800460, 23800480, 23800500, 23800501, 23800520, 23800557, 23800620, 23800690, 23800760, 23800900, 23900200, 23900600, 23900620, 24300042, 24300202, 24300304, 24300406, 24300600, 24500100, 24700109, 24700200, 24700302, 24700325, 24700335, 24700404, 24709089, 24900108, 25100100, 25100300, 25300200, 25463, 25700100, 25700300, 25700500, 25700600, 25800100, 25800177, 25800200, 25800440, 25800500, 25800600, 25800800, 25830056, 25900700, 26100100, 26100140, 26100200, 26358

## Extract River Gauge Data and Save to DataFrame

In [4]:
def extract_data_from_url(url):
    """
    Extracts specified data from a river gauge (pegel) URL and returns a DataFrame.

    Parameters:
    url (str): The URL of the webpage containing river gauge data.

    Returns:
    pandas.DataFrame: DataFrame containing extracted data fields such as 'Messstellenname', 'Messstellennummer', 
                      'Gewässer', 'Kilometer an der Wasserstraße', 'Koordinate', 'Rechtswert_Hochwert'.
                      Returns None if the request fails or no data is found.
    """
    # Make a GET request to the website
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table with class "stammdaten_width722"
        table = soup.find('table', {'class': 'stammdaten_width722'})

        # Extract data from the table into a dictionary
        data = {
            'Messstellenname': '',
            'Messstellennummer': '',
            'Gewässer': '',
            'Kilometer an der Wasserstraße': '',
            'Koordinate': '',
            'Rechtswert_Hochwert': ''
        }

        for row in table.find_all('tr'):
            columns = row.find_all(['td', 'th'])
            if len(columns) == 2:
                key = columns[0].get_text(strip=True)
                value = columns[1].get_text(strip=True)
                if 'Messstellenname' in key:
                    data['Messstellenname'] = value
                elif 'Messstellennummer' in key:
                    data['Messstellennummer'] = value
                elif 'Gewässer' in key:
                    data['Gewässer'] = value
                elif 'Kilometer an der Wasserstraße' in key:
                    data['Kilometer an der Wasserstraße'] = value

        # Find the Koordinate cell
        koordinate_cell = soup.find('td', text=lambda text: text and 'Koordinate' in text)
        if koordinate_cell:
            koordinate_text = koordinate_cell.get_text(strip=True)
            data['Koordinate'] = koordinate_text

        # Find the Rechtswert-Hochwert cell (lat-long)
        rechtswert_hochwert_cell = soup.find('td', text=lambda text: text and 'Rechtswert' in text)
        if rechtswert_hochwert_cell:
            rechtswert_hochwert_text = rechtswert_hochwert_cell.get_text(strip=True)
            data['Rechtswert_Hochwert'] = rechtswert_hochwert_text

        # Create a DataFrame from the dictionary
        df = pd.DataFrame(data, index=[0])

        return df
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None


In [5]:
# Example usage:
# List of URLs to extract data from
urls = []
for pegel_nummer in pegelnummern:
    url = "https://www.pegelonline.wsv.de/gast/stammdaten?pegelnr=" + str(pegel_nummer)
    urls.append(url)

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate through the list of URLs and append the extracted data to the result DataFrame
for url in urls:
    data_df = extract_data_from_url(url)
    if data_df is not None:
        result_df = pd.concat([result_df, data_df], ignore_index=True)

C:\Users\Omar.Seleem\AppData\Local\Temp\ipykernel_12480\1134851915.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  koordinate_cell = soup.find('td', text=lambda text: text and 'Koordinate' in text)
C:\Users\Omar.Seleem\AppData\Local\Temp\ipykernel_12480\1134851915.py:55: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  rechtswert_hochwert_cell = soup.find('td', text=lambda text: text and 'Rechtswert' in text)


In [7]:
# Extract Rechtswert and Hochwert (lat and long) using regex
result_df[['Rechtswert', 'Hochwert']] = result_df['Rechtswert_Hochwert'].str.extract(r'Rechtswert:\s*([\d.]+).*Hochwert:\s*([\d.]+)')

In [8]:
result_df

,Messstellenname,Messstellennummer,Gewässer,Kilometer an der Wasserstraße,Koordinate,Rechtswert_Hochwert,Rechtswert,Hochwert
0,INGOLSTADT LUITPOLDSTRASSE,10046105,DONAU,2458.3 km,"Koordinate (Gauss-Krüger 4, Bessel 1841, DHDN)",Rechtswert: 4457907.0; Hochwert: 5402367.0,4457907.0,5402367.0
1,KELHEIMWINZER,10054500,DONAU,2409.7 km,"Koordinate (Zone 32N, ETRS 1989, UTM)",Rechtswert: 714917.85; Hochwert: 5421780.68,714917.85,5421780.68
2,OBERNDORF,10056302,DONAU,2397.4 km,"Koordinate (Zone 32N, ETRS 1989, UTM)",Rechtswert: 720740.48; Hochwert: 5425934.46,720740.48,5425934.46
3,REGENSBURG EISERNE BRÜCKE,10061007,DONAU,2379.3 km,"Koordinate (Zone 32N, ETRS 1989, UTM)",Rechtswert: 726762.99; Hochwert: 5434466.27,726762.99,5434466.27
4,SCHWABELWEIS,10062000,DONAU,2376.5 km,"Koordinate (Zone 32N, ETRS 1989, UTM)",Rechtswert: 729447.57; Hochwert: 5434824.34,729447.57,5434824.34
...,...,...,...,...,...,...,...,...
661,SANKT ALDEGUND UP,26900100,MOSEL,78.1 km,"Koordinate (Zone 32N, ETRS 1989, UTM)",Rechtswert: 366520.0; Hochwert: 5550231.0,366520.0,5550231.0
662,TRIER UP,26500100,MOSEL,195.3 km,"Koordinate (Gauss-Krüger 2, Bessel 1841, DHDN)",Rechtswert: 2545235.0; Hochwert: 5510809.0,2545235.0,5510809.0
663,WINCHERINGEN,26100140,MOSEL,222.2 km,"Koordinate (Gauss-Krüger 3, Bessel 1841, DHDN)",Rechtswert: 3312482.165; Hochwert: 5499907.91,3312482.165,5499907.91
664,WINTRICH UP,26700400,MOSEL,141.1 km,"Koordinate (Gauss-Krüger 3, Bessel 1841, DHDN)",Rechtswert: 3352482.0; Hochwert: 5528604.0,3352482.0,5528604.0


In [77]:
# Save the dataframe to excel
result_df.to_excel('result_data.xlsx', index=False)


You can also save them to a shapefile using GeoPandas, ensuring they do not share the same coordinate system.